In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [6]:
max_features=10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

maxlen = 500
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)





In [9]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', test_accuracy)
# Save the model
model.save('rnn_imdb_model.h5')

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 115ms/step - accuracy: 0.5965 - loss: 0.7192 - val_accuracy: 0.6464 - val_loss: 0.6231
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 115ms/step - accuracy: 0.6058 - loss: 19594.1191 - val_accuracy: 0.6114 - val_loss: 0.7260
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 115ms/step - accuracy: 0.6995 - loss: 0.5754 - val_accuracy: 0.7034 - val_loss: 0.5657
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 114ms/step - accuracy: 0.7786 - loss: 0.4754 - val_accuracy: 0.7506 - val_loss: 0.5061
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 117ms/step - accuracy: 0.8260 - loss: 0.3973 - val_accuracy: 0.7804 - val_loss: 0.4688
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 116ms/step - accuracy: 0.8209 - loss: 0.8366 - val_accuracy: 0.6924 - val_loss: 0.5677
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 121ms/step - accuracy: 0.6615 - loss: 172918980608.0000 - val_accuracy: 0.6048 - val_loss: 0.6705
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 118ms/step - accuracy: 0

Test accuracy: 0.7659599781036377


In [ ]:
# decode reviews
def decode_review(text):
    word_index = imdb.get_word_index()
    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    decoded = ' '.join([reverse_word_index.get(i - 3, '?') for i in text])
    return decoded

# Preprocess input text
def preprocess_text(text, maxlen=500):
    word_index = imdb.get_word_index()
    text = text.lower().split()
    text = [word_index.get(word, 0) for word in text]
    return sequence.pad_sequences([text], maxlen=maxlen)

# Predict sentiment of a new review
def predict_sentiment(review):
    processed_review = preprocess_text(review)
    prediction = model.predict(processed_review)
    sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
    return sentiment

# Example usage

review = "This movie was fantastic! I loved the plot and the acting."
sentiment = predict_sentiment(review)
print("Sentiment:", sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Sentiment: positive
